In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults = "../../results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
import re

evalTime = 25.

path2file_ = r'getsrivssriset/{}_1476_{}_10to25_20mn_0s_64/'
setIds = [5,44,61]

In [8]:
def getValOld(mypath, filename, header="infer",names = None):
    fullpath = pathresults +mypath+filename
    if names is None:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header).dropna(how= "all", axis=1)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names).dropna(how= "all", axis=1)
    return cst

def getVal(mypath, filename,dtype=float,extension='.txt', header="infer",names = None):
    fullpath = pathresults +mypath+filename
    data_file_delimiter = ","
    if names is None:
        largest_column_count = 0
        with open(fullpath, 'r') as temp_f:
            lines = temp_f.readlines()
        last_line = lines[-1].strip()
        #print('last_line',last_line)
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)
        largest_column_count = len(array_from_last_line)
        names = [i for i in range(0, largest_column_count)]
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, 
                          header = header, names = names)
        #print('cst',cst.shape)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names)
    return cst

def getTimeLine(path2file, evalTime):
    time = pd.read_csv(pathresults + path2file + "time.txt", names = ["time","Qlight"])["time"][1:] # because we have twice the initial value
    time_diff = abs(np.array(time)-evalTime)
    timeLineNeg = len(time_diff) - np.where(time_diff == min(time_diff))[0][0] 
    timeLinePos = len(time_diff) -timeLineNeg
    print('timeLine',timeLineNeg,timeLinePos, min(time_diff) )
    return timeLineNeg, timeLinePos


def get_line_X_as_numpy_array(file_path, timeLine,pos = False, dtype = float, verbose = False):
    with open(pathresults +file_path, 'r') as file:
        # Read all lines
        lines = file.readlines()

        # Check if there are any lines in the file
        if not lines:
            raise ValueError("The file is empty")

        # Get the last line
        
        try:
            if pos:
                last_line = lines[timeLine-1].strip()
            else:
                last_line = lines[-timeLine-1].strip()
        except:
            print('timeLine',file_path, len(lines)-timeLine, len(lines))
            raise Exception

        # Split the last line by commas and convert to NumPy array
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)
        if verbose:
            print(file_path, len(array_from_last_line), end=", ")

        return array_from_last_line

def get_last_line_as_numpy_array(file_path, timeLine,pos = False, dtype = float, verbose = False):
    with open(pathresults +file_path, 'r') as file:
        # Read all lines
        lines = file.readlines()

        # Check if there are any lines in the file
        if not lines:
            raise ValueError("The file is empty")

        # Get the last line
        
        try:
            if pos:
                last_line = lines[timeLine-1].strip()
            else:
                last_line = lines[-timeLine-1].strip()
        except:
            print('timeLine',file_path, len(lines)-timeLine, len(lines))
            raise Exception

        # Split the last line by commas and convert to NumPy array
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)
        if verbose:
            print(file_path, len(array_from_last_line), end=", ")

        return array_from_last_line

def getCylIdx(path2file, timeLinePos):
    fileName = "rhizoSegsId.txt"
    fullpath = path2file+fileName
    rhizoSegsId = get_last_line_as_numpy_array(fullpath, timeLinePos,pos=True, dtype = int, verbose = True)
    print('rhizoSegsId',max(rhizoSegsId))
    return rhizoSegsId
    
def list_files_with_prefix(folder_path, prefix):
    file_names = []
    for file_name in os.listdir(pathresults+folder_path):
        if file_name.startswith(prefix):
            file_names.append(file_name)
    return file_names

# cylinder max konz (per cell)
def getData_(cid,gId,path2file, timeLineNeg):
    fileName = "cyl_val/Cyl_cellVol_"+str(gId)+".txt"
    cVol = get_last_line_as_numpy_array(path2file+fileName, timeLineNeg,pos= False, dtype = float)
    if cid == volIdx:
        return cVol
    elif cid <= 9:
        if cid <= 2:# == 0:
            fileName = "cyl_val/Cyl_watercontent_"+str(gId)+".txt"
            theta = get_last_line_as_numpy_array(path2file+fileName, timeLineNeg,pos= False,  dtype = float)
            cVol *= theta#cm3 scv to cm3 water
            if cid == 0:
                return cVol
        fileName = "cyl_val/Cyl_content"+str(cid)+"_"+str(gId)+".txt"
        Q1 = get_last_line_as_numpy_array(path2file+fileName, timeLineNeg, pos= False, dtype = float)
        konz = Q1/cVol #mol/cm3 or cm3/cm3
        return konz
    elif cid == coordId:
        fileName = "cyl_val/Cyl_coord_"+str(gId)+".txt"
        Q1 = get_last_line_as_numpy_array(path2file+fileName, timeLineNeg, pos= False, dtype = float)
        return Q1
    raise Exception

nKonz = 9
dictXYZ = {9:'X',10:'Y',11:'Z'}
volIdx=12
lenIdx=13
stIdx=14
orgIdId=15
relLenId=16
coordId = 17

nToGet = coordId+1#elements + 3d coordinates of y , node+vol+len+st 

# baseline_1080_19_10to25_20mn_0s_128
scenarios = ["earlyDry", "baseline", "lateDry"]
exceptPset = []#[('baseline','19'),('baseline','47'),('baseline','83')]
#result_list_compExcept = [path2file.format(scenario, str(setId)) for scenario, setId in exceptPsets]
result_list_comp = [path2file_.format(scenario, str(setId)) for scenario in scenarios for setId in setIds if (scenario, str(setId)) not in exceptPset ]

numPset = len(result_list_comp)
allPSets = [ i for i in range(numPset) if i not in exceptPset]

def getData():
    
    GiniAll = [[[] for i in range(numPset)] for ii in range(nToGet)]
    
    for pSet in allPSets:
        print(pSet,end =", ")
        if True:

        #try:
            #path2file = 'none_55_'+str(pSet)+'_10to11_20mn_0s_5/'
            path2file =result_list_comp[pSet]# "forEGU"+scenarios[pSet]+"_1440_76_10to25_20mn_0s_128/"

            time = pd.read_csv(pathresults + path2file + "time.txt", names = ["time","Qlight"])["time"][1:] # because we have twice the initial value
            timemax = int((max(time))*10)/10
            print('timemax',timemax, timemax==25)
            
            if(timemax==25):
                timeLineNeg, timeLinePos = getTimeLine(path2file, evalTime)
                rr = getCylIdx(path2file, timeLinePos)

                rr.sort()
                print('(',len(rr),')',end =", ")

                for cid in range(nKonz):
                    Ginits = np.array([getData_( cid, 
                                                gId,path2file, timeLineNeg) for gId in rr])          
                    GiniAll[cid][pSet] = Ginits
                for cid in range(nKonz,volIdx):
                    nodes_ = get_last_line_as_numpy_array(path2file+"nodes_"+dictXYZ[cid]+".txt", timeLinePos,pos = True, dtype =float)
                    GiniAll[cid][pSet] = nodes_[rr+1]#seg idx to y-idx
                #cid = nToGet -1
                Ginits = np.array([getData_( volIdx, gId,path2file, timeLineNeg) for gId in rr])
                GiniAll[volIdx][pSet] = Ginits
                
                Ginits = np.array([getData_( coordId, gId,path2file, timeLineNeg) for gId in rr])
                GiniAll[coordId][pSet] = Ginits
                
                
                lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", timeLinePos,pos = True, dtype =float, verbose = True)
                GiniAll[lenIdx][pSet] = lens[rr]
                sts = get_last_line_as_numpy_array(path2file+"subTypes"+".txt", timeLinePos,pos = True, dtype =float, verbose = True)
                assert len(sts) == len(lens)
                GiniAll[stIdx][pSet] = sts[rr]

                orgidPerNode = get_last_line_as_numpy_array(path2file+"orgidPerNode.txt", timeLinePos,pos = True, dtype =int, verbose = True)
                globalNodeId = get_last_line_as_numpy_array(path2file+"globalNodeId.txt", timeLinePos,pos = True, dtype =int, verbose = True)
                parentNidx = get_last_line_as_numpy_array(path2file+"parentNidx.txt", timeLinePos, pos = True,dtype =int, verbose = True)
                ot_orgs = get_last_line_as_numpy_array(path2file+"ot_orgs.txt", timeLinePos,pos = True, dtype =int, verbose = True)
                id_orgs = get_last_line_as_numpy_array(path2file+"id_orgs.txt", timeLinePos, pos = True,
                                                       dtype =int, verbose = True)
                id_roots = id_orgs[np.where(ot_orgs==2)]

                org2basenode = dict(zip(id_orgs, parentNidx))
                #org2ot = dict(zip(id_orgs, ot_orgs))
                isRoot = np.array([oo in id_roots for oo in orgidPerNode])

                orgidPerRoot = orgidPerNode[np.where(isRoot)]
                globalNodeIdroot =  globalNodeId[np.where(isRoot)]        
                segIdxRoot = np.array([rr for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]]) -1
                orgIdxRoot = np.array([orgidPerRoot[_] for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]])
                #segIdx2ot = dict(zip(segIdxRoot,orgIdxRoot))

                relLens = np.zeros(len(segIdxRoot))
                #lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float)
                for myorg in orgIdxRoot:
                    thensegs = np.where(orgIdxRoot == myorg)#segIdxRoot[] 
                    lenSegs = GiniAll[lenIdx][pSet]
                    relLens[thensegs] = np.cumsum(lenSegs[thensegs])


                GiniAll[orgIdId][pSet] = orgIdxRoot
                GiniAll[relLenId][pSet] = relLens

            #except:
            #    print('jump',end =", ")

    return GiniAll #cid pSet rr
    

In [9]:

GiniAll = getData()

0, timemax 25.0 True
timeLine 1 1079 4.227729277772596e-13
getsrivssriset/earlyDry_1476_5_10to25_20mn_0s_64/rhizoSegsId.txt 457, rhizoSegsId 514
( 457 ), getsrivssriset/earlyDry_1476_5_10to25_20mn_0s_64/length_Segs.txt 515, getsrivssriset/earlyDry_1476_5_10to25_20mn_0s_64/subTypes.txt 515, getsrivssriset/earlyDry_1476_5_10to25_20mn_0s_64/orgidPerNode.txt 515, getsrivssriset/earlyDry_1476_5_10to25_20mn_0s_64/globalNodeId.txt 515, getsrivssriset/earlyDry_1476_5_10to25_20mn_0s_64/parentNidx.txt 217, getsrivssriset/earlyDry_1476_5_10to25_20mn_0s_64/ot_orgs.txt 217, getsrivssriset/earlyDry_1476_5_10to25_20mn_0s_64/id_orgs.txt 217, 1, timemax 25.0 True
timeLine 1 1079 4.227729277772596e-13
getsrivssriset/earlyDry_1476_44_10to25_20mn_0s_64/rhizoSegsId.txt 458, rhizoSegsId 515
( 458 ), getsrivssriset/earlyDry_1476_44_10to25_20mn_0s_64/length_Segs.txt 516, getsrivssriset/earlyDry_1476_44_10to25_20mn_0s_64/subTypes.txt 516, getsrivssriset/earlyDry_1476_44_10to25_20mn_0s_64/orgidPerNode.txt 516, 

In [22]:
GiniAll_ =GiniAll

In [21]:

g_cylindx = [[] for i in range(len(allPSets))]
for pp in range(len(allPSets)):
    numSeg = len(GiniAll[orgIdId][pp])
    g_cylindx[pp] = np.array([rr for rr in range(numSeg)] )


In [23]:
GiniAll.append(g_cylindx)
df = pd.DataFrame(GiniAll)

In [24]:
df[0]

0     [[0.0006817294260771512, 0.000959653428732712,...
1     [[2.8486815735298076e-06, 2.84867947627166e-06...
2     [[0.0005915265795148129, 0.0005915056695894194...
3     [[1.1226480088381245e-07, 1.118811300196757e-0...
4     [[9.194713284194991e-06, 9.194932855723927e-06...
5     [[1.0808634899075511e-07, 1.0805647313594791e-...
6     [[9.495175223481356e-06, 9.492576502558687e-06...
7     [[6.567094140939179e-05, 6.567087197882797e-05...
8     [[4.24630068178538e-05, 4.221540397626501e-05,...
9     [-0.29843419619737527, 0.04392226409042643, -0...
10    [-0.3048545919249007, -0.540902871740023, -0.6...
11    [-1.2044339159509272, -2.113869600150491, -2.9...
12    [[0.00320187148889538, 0.004507194255000854, 0...
13    [1.0, 0.9999999999999999, 1.0, 1.0000000000000...
14    [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
15    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
16    [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...
17    [[0.0546613694441331, 0.06485324293618723,

In [25]:
df = pd.DataFrame(GiniAll)
df.to_pickle('./4pallcellat25.pkl')

In [ ]:
# BIN

In [12]:
dataInput =  [(scenario, str(setId)) for scenario in scenarios for setId in setIds if (scenario, str(setId)) not in exceptPset ]

In [19]:
# get param set

outputvalsname =  ['wat','cs','cl','coa','cod','cca','ccd','co2','css2','yX','yY','yZ','vol','lenSeg','st','orgId','relLen', 'coord']
assert len(outputvalsname) == nToGet 
column_names = outputvalsname + ['scenario']
df = pd.DataFrame(columns=column_names)

for pset, din in enumerate(dataInput):#range(numPset):
    N = GiniAll[0][pset].shape[0]
    #assert N == 232
    rows_df = pd.DataFrame()

    for jj, outVn in enumerate(outputvalsname):
        rows_df[outVn] = GiniAll[jj][pset]
        #print('adding',jj,outVn,duplicated_rows_df.shape,len(GiniAll[jj][pset]))
    rows_df['scenario'] = din[0]
    rows_df['pSet'] = din[1]

    df = pd.concat([df, rows_df], ignore_index=True)
df['co'] = df['coa']+df['cod']
df['cc'] = df['cca']+df['ccd']

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [30]:
df.to_pickle('./4paramTemp.pkl')

In [1]:
df

NameError: name 'df' is not defined

In [15]:
nToGet

18

In [16]:
len(  ['wat','cs','cl','coa','cod','cca','ccd','co2','yX','yY','yZ','vol','lenSeg','st','orgId','relLen'])

16